# Proces uczenia sieci

## The data

In [1]:
# Na początku definiujemy wszystkie biblioteki i narzędzia wykorzystywane do definicji sieci
# w tej fazie kodu korzystamy z  bibliotek keras i tensroflow
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import pylab
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import warnings
from tensorflow.python.keras import backend as k
warnings.filterwarnings("ignore")

D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dty

In [2]:
#rozpoczynamy liczenie czasu, w celu określenia długości trwania programu
import time
start = time.time()

In [3]:
#wczytuje dane do kodu, w celu określenia danych wejściowych i wyjściowych
# dane wczytuję w postaci macierzowej
dane = pd.read_csv(r'C:\Users\\Lukasz K\\Desktop\\Praca/daneT800-1500K.csv', sep = ";")
array = dane.values
X = array[:,0:4]
X1 = array[:,5]
X2 = array[:,0:7]

In [4]:
X

array([[8.0e+02, 2.0e-01, 2.0e+00, 3.0e+00],
       [8.0e+02, 2.0e-01, 2.0e+00, 3.0e+00],
       [8.0e+02, 2.0e-01, 2.0e+00, 3.0e+00],
       ...,
       [1.4e+03, 8.0e-01, 2.5e+01, 8.0e+00],
       [1.4e+03, 8.0e-01, 2.5e+01, 8.0e+00],
       [1.4e+03, 8.0e-01, 2.5e+01, 8.0e+00]])

In [5]:
X1

array([0.  , 0.03, 0.06, ..., 0.24, 0.27, 0.3 ])

In [6]:
X2

array([[8.00000000e+02, 2.00000000e-01, 2.00000000e+00, ...,
        1.08703561e+00, 0.00000000e+00, 0.00000000e+00],
       [8.00000000e+02, 2.00000000e-01, 2.00000000e+00, ...,
        1.08703561e+00, 3.00000000e-02, 3.30353722e+00],
       [8.00000000e+02, 2.00000000e-01, 2.00000000e+00, ...,
        1.08703561e+00, 6.00000000e-02, 6.76408966e+00],
       ...,
       [1.40000000e+03, 8.00000000e-01, 2.50000000e+01, ...,
        5.92302719e-02, 2.40000000e-01, 6.36851235e+04],
       [1.40000000e+03, 8.00000000e-01, 2.50000000e+01, ...,
        5.92302719e-02, 2.70000000e-01, 7.46325305e+04],
       [1.40000000e+03, 8.00000000e-01, 2.50000000e+01, ...,
        5.92302719e-02, 3.00000000e-01, 8.67351486e+04]])

In [7]:
#wypisuję dane
dane.describe()

,T,VF,Tr,l_tpb,E,i,Cd
count,77616.000000,77616.000000,77616.000000,77616.000000,77616.000000,77616.000000,77616.000000
mean,1100.000000,0.500000,13.500000,5.500000,0.241622,0.150000,10057.806008
std,200.001288,0.200001,6.922231,1.707836,0.217229,0.094869,20500.200018
min,800.000000,0.200000,2.000000,3.000000,0.029615,0.000000,0.000000
25%,900.000000,0.300000,7.750000,4.000000,0.100339,0.060000,80.596965
50%,1100.000000,0.500000,13.500000,5.500000,0.167297,0.150000,1420.647118
75%,1300.000000,0.700000,19.250000,7.000000,0.307292,0.240000,10104.607644
max,1400.000000,0.800000,25.000000,8.000000,2.174071,0.300000,306611.420013


In [8]:
#wyrzucam z zestawu rekordów ostatnią kolumnę, stanie się ona danymi wyjściowymi dla sieci
#pozostałe 6 kolumn to dane wejściowe do sieci
x_data = dane.drop(['Cd'],axis=1)
x_data

,T,VF,Tr,l_tpb,E,i
0,800,0.2,2,3,1.087036,0.00
1,800,0.2,2,3,1.087036,0.03
2,800,0.2,2,3,1.087036,0.06
3,800,0.2,2,3,1.087036,0.09
4,800,0.2,2,3,1.087036,0.12
...,...,...,...,...,...,...
77611,1400,0.8,25,8,0.059230,0.18
77612,1400,0.8,25,8,0.059230,0.21
77613,1400,0.8,25,8,0.059230,0.24
77614,1400,0.8,25,8,0.059230,0.27


In [9]:
#wypisuję rekordy danych wyjściowych
y_val = dane[['Cd']].as_matrix()
y_val

array([[0.00000000e+00],
       [3.30353722e+00],
       [6.76408966e+00],
       ...,
       [6.36851235e+04],
       [7.46325305e+04],
       [8.67351486e+04]])

In [10]:
dane.describe()

,T,VF,Tr,l_tpb,E,i,Cd
count,77616.000000,77616.000000,77616.000000,77616.000000,77616.000000,77616.000000,77616.000000
mean,1100.000000,0.500000,13.500000,5.500000,0.241622,0.150000,10057.806008
std,200.001288,0.200001,6.922231,1.707836,0.217229,0.094869,20500.200018
min,800.000000,0.200000,2.000000,3.000000,0.029615,0.000000,0.000000
25%,900.000000,0.300000,7.750000,4.000000,0.100339,0.060000,80.596965
50%,1100.000000,0.500000,13.500000,5.500000,0.167297,0.150000,1420.647118
75%,1300.000000,0.700000,19.250000,7.000000,0.307292,0.240000,10104.607644
max,1400.000000,0.800000,25.000000,8.000000,2.174071,0.300000,306611.420013


In [11]:
#skaluję dane przed dodaniem ich do algorytmu
#dane muszą spełniać określone warunki, np mieścić się w odpowiednim zakresie
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scalerx = scaler.fit(x_data)
x_data = scaler.transform(x_data)
x_data=pd.DataFrame(x_data, columns=['T','VF',  'Tr','l_tpb','E',  'i'])
x_data

,T,VF,Tr,l_tpb,E,i
0,0.0,0.0,0.0,0.0,0.493095,0.0
1,0.0,0.0,0.0,0.0,0.493095,0.1
2,0.0,0.0,0.0,0.0,0.493095,0.2
3,0.0,0.0,0.0,0.0,0.493095,0.3
4,0.0,0.0,0.0,0.0,0.493095,0.4
...,...,...,...,...,...,...
77611,1.0,1.0,1.0,1.0,0.013810,0.6
77612,1.0,1.0,1.0,1.0,0.013810,0.7
77613,1.0,1.0,1.0,1.0,0.013810,0.8
77614,1.0,1.0,1.0,1.0,0.013810,0.9


In [12]:
#podział danych na testowe i walidacyjne
#dane testowe stanowią w tym przypadku 30% zbioru, wartość tą można zmieniać
#random_state służy do generowania zestawów liczb do uczenia i sprawdzania, przy powtarzaniu sekwencji ta sama wartość gwarantuje ten sam zestaw
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

In [13]:
#włożenie danych do macierzy
X_test1 = X_test.as_matrix()
X_tst = scalerx.inverse_transform(X_test1)
X_tst = pd.DataFrame(X_tst)

In [14]:
#wypisanie ilości wejść
input_shape=(X_train.shape[1],)
input_shape

(6,)

In [15]:
import tensorflow as tf
#przejscie na tf_feature
T = tf.feature_column.numeric_column('T')
E = tf.feature_column.numeric_column('E')
l_tpb = tf.feature_column.numeric_column('l_tpb')
Tr = tf.feature_column.numeric_column('Tr')
i = tf.feature_column.numeric_column('i')

In [16]:
#deifiniowanie modelu i określenie kształtu sieci
#określenie ilości warstw ukrytych, wielkości, funkcji aktywacyjnych
#w przypadku tego rodzaju funkcji wykorzystuje się relu, elu, selu, leaky_relu
def build_model():
    model = keras.Sequential([
    keras.layers.Dense(6, activation=tf.nn.relu,
                       input_shape=(X_train
                                .shape[1],)),
     
    keras.layers.Dense(15, activation=tf.nn.relu),
    keras.layers.Dense(19, activation=tf.nn.relu),
    keras.layers.Dense(24, activation=tf.nn.relu),
    keras.layers.Dense(19, activation=tf.nn.relu),
    keras.layers.Dense(15, activation=tf.nn.relu),    
    
    keras.layers.Dense(1)
  ])

    optimizer = tf.train.RMSPropOptimizer(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
    return model

model = build_model()
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 42        
_________________________________________________________________
dense_1 (Dense)              (None, 15)                105       
_________________________________________________________________
dense_2 (Dense)              (None, 19)                304       
_________________________________________________________________
dense_3 (Dense)              (None, 24)                480       
_________________________________________________________________
dense_4 (Dense)              (None, 19)                475       
_________________________________________________________________
dense_5 (Dense)              (None, 15)                300       
____

In [17]:
#podczas budowy modelu po każdej zakończonej epoce wypisywana zostanie kropka
#można użyć logging.info() jednak dla mnie nie ma różnicy
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

    #definiuję maksymalną liczbę epok wykonaną przez algorytm
EPOCHS = 500

# przechowywuję dane z uczenia w celu nakreślenia wykresów
history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.27, verbose=0,
                    callbacks=[PrintDot()])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................

In [18]:
#z modelu wytworzonego w komórce wcześniej, buduję model rzeczywisty
#działania prowadzone są w sposób równoległy, ten etap uzupełnia komórkę wyżej
model = build_model()

#  callbacks to zestaw funkcji, które należy zastosować na poszczególnych etapach procedury treningowej. 
#  Możesz użyć callbacks, aby uzyskać widok stanów wewnętrznych i statystyk modelu podczas treningu.
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
#model przerywa symulację kiedy 40 kolejnych wyników powtarza się, zaznacza się że wtedy sieć jest przeszkolona i gotowa do użytkowania

history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,)

In [19]:
#deifiniowanie modelu i określenie kształtu sieci
#określenie ilości warstw ukrytych, wielkości, funkcji aktywacyjnych
#w przypadku tego rodzaju funkcji wykorzystuje się relu, elu, selu, leaky_relu
def build_model1():
    
    model1 = keras.Sequential([
    keras.layers.Dense(6, activation=tf.nn.relu,
                       input_shape=(X_train
                                .shape[1],)),
     
    keras.layers.Dense(18, activation=tf.nn.relu),
    keras.layers.Dense(22, activation=tf.nn.relu),
    keras.layers.Dense(30, activation=tf.nn.relu),
    keras.layers.Dense(25, activation=tf.nn.relu),
    keras.layers.Dense(15, activation=tf.nn.relu),    
    
    keras.layers.Dense(1)
  ])

    optimizer = tf.train.RMSPropOptimizer(0.001)
    model1.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
    return model1

model1 = build_model1()
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_15 (Dense)             (None, 18)                126       
_________________________________________________________________
dense_16 (Dense)             (None, 22)                418       
_________________________________________________________________
dense_17 (Dense)             (None, 30)                690       
_________________________________________________________________
dense_18 (Dense)             (None, 25)                775       
_________________________________________________________________
dense_19 (Dense)             (None, 15)                390       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                

In [20]:
EPOCHS = 500
history1 = model1.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.27, verbose=0,
                    callbacks=[PrintDot()])


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................

In [ ]:
model1 = build_model1()


early_stop1 = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)

history1 = model1.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[early_stop, PrintDot()])


....................................................................................................
....................................................................................................
........................................................................

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
import matplotlib.ticker as ticker

def plot_porownanie(history, history1):
    


    plt.figure(facecolor='white', figsize = [10,5])
    
  
    error = np.array(history.history['mean_absolute_error'])
    errorval = np.array(history.history['val_mean_absolute_error'])
    
    error1 = np.array(history1.history['mean_absolute_error'])
    errorval1 = np.array(history1.history['val_mean_absolute_error'])

  
    
    fig, ax = plt.subplots(figsize=[10, 5], facecolor='white', sharex=True)

    
  
    ax.plot(history.epoch, error, 'b--',
           label='Train Loss')
    ax.plot(history.epoch, errorval, 'g',
           label = 'Val loss')
    
    ax.plot(history1.epoch, error1, 'k--',
           label='Train Loss1')
    ax.plot(history1.epoch, errorval1, 'r',
           label = 'Val loss1')
  
    axins = zoomed_inset_axes(ax, 3, loc=1)
    axins.plot(history.epoch, error, 'b--')
    axins.plot(history.epoch, errorval, 'g')
    axins.plot(history1.epoch, error1, 'k--')
    axins.plot(history1.epoch, errorval1, 'r')
    
    if history.epoch > history1.epoch :
        
        x1, x2, y1, y2 = history.epoch[-50], history.epoch[-1], errorval[-1]-500, errorval[-1]+500 
        axins.set_xlim(x1, x2)
        axins.set_ylim(y1, y2)
        
    else:
        x1, x2, y1, y2 = history1.epoch[-50], history1.epoch[-1], errorval1[-1]-500, errorval1[-1]+500
        axins.set_xlim(x1, x2) 
        axins.set_ylim(y1, y2)
        
    
    
    axins.tick_params(axis='both', which='both', labelsize=8, labelcolor = 'k', size = 5)
    
    
    xticklabels = x1, (x2+x1)/2, x2
    yticklabels = y1, (y2+y1)/2, y2
    axins.xaxis.set_major_locator(ticker.FixedLocator(xticklabels))
    axins.yaxis.set_major_locator(ticker.FixedLocator(yticklabels))
    
    axins.yaxis.set_visible(True)
    ax.legend(['Dane uczące sieć o warstwach ukrytych 15-19-24-19-15', 'Dane walidacyjne sieć o warstwach ukrytych 15-19-24-19-15'
               , 'Dane uczące sieć o warstwach ukrytych 18-22-30-25-15', 'Dane walidacyjne o warstwach ukrytych 18-22-30-25-15'],
              loc = 2, frameon= True)
    mark_inset(ax, axins, loc1=3, loc2=4, lw='0.5')
    
    

    ax.set_xlabel('Liczba epok', fontsize=15)
    ax.set_ylabel('Średni błąd' , fontsize=15)
    ax.grid(True)

In [ ]:
    plot_porownanie(history, history1, history2)
    plt.savefig(r'C:\Users\Lukasz K\Desktop\\wykresy do pracy/annzamierzone.png')

In [ ]:
#definicja błędów
[loss, mae] = model.evaluate(X_test, y_test, verbose=0)
print("Testing set Mean Abs Error: {:7.2f}".format(mae ))

[loss, mae] = model1.evaluate(X_test, y_test, verbose=0)
print("Testing set Mean Abs Error: {:7.2f}".format(mae ))

In [ ]:
#zapis modelów w celu późniejszego użycia
#model
model_json = model.to_json()
with open(r"C:\Users\\Lukasz K\\Desktop\\Praca\\wykresy/porownanie1.json", "w") as json_file:
    json_file.write(model_json)
#wagi
model.save_weights(r"C:\\Users\Lukasz K\\Desktop\\Praca\\wykresy/porownanie1.h5")
print("Saved model to disk")

In [ ]:
#model
model_json = model1.to_json()
with open(r"C:\\Users\\Lukasz K\\Desktop\\Praca\\wykresy/porownanie2.json", "w") as json_file:
    json_file.write(model_json)
#wagi
model1.save_weights(r"C:\\Users\\Lukasz K\\Desktop\Praca\\wykresy/porownanie2.h5")
print("Saved model to disk")

In [ ]:
test_predictions = model.predict(X_test).flatten()
test_predictions1 = model1.predict(X_test).flatten()

In [ ]:
from sklearn.metrics import mean_squared_error
MSE1 = mean_squared_error(y_test,test_predictions)**0.5
MSE2 = mean_squared_error(y_test,test_predictions1)**0.5

print ('MSE1 = %s, MSE2 =%s' %(MSE1, MSE2))

In [ ]:
from sklearn.metrics import r2_score
r1 = r2_score(y_test, test_predictions)
r2 = r2_score(y_test, test_predictions1)
print ('r1 = %s, r2 =%s' %(r1, r2))

In [ ]:
end = time.time()
print(end - start)

In [ ]:
open

In [ ]:
import json
from pprint import pprint
from keras.models import model_from_json

json_file = open('modelVF1.2.7.1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("modelVF1.2.7.1.h5")
print("Loaded model from disk")
optimizer = tf.train.RMSPropOptimizer(0.001)

loaded_model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])


                     
[loss, mae] = loaded_model.evaluate(X_test.values, y_test, verbose=0)

print("Testing set Mean Abs Error: {:7.2f}".format(mae ))